In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# import required packages
import pandas as pd
import numpy as np

# preprocessing
from sklearn.model_selection import train_test_split

# models
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV 

# metrics
import time
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.metrics import recall_score, roc_auc_score

# plots
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')